In [5]:
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
#funcion para consultar en la pagina de farmacity
def obtener_info_farmacity(nombre_producto):
    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()

    # Abre la URL de Farmacity usando un f-string para insertar el nombre del producto
    url = f"https://www.farmacity.com/{nombre_producto}?_q={nombre_producto}&map=ft"
    driver.get(url)
    
    # Encuentra los elementos que contienen el título de los productos
    titles = driver.find_elements(By.XPATH, '//span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body"]')

    # Encuentra los elementos que contienen el precio de los productos
    prices = driver.find_elements(By.XPATH, '//span[@class="vtex-product-price-1-x-currencyContainer"]')
       
    # Imprime la cantidad de elementos encontrados
    print(f"Se encontraron {len(titles)} elementos")
    
    productos = []
    # Iterar sobre los elementos y obtener los títulos y precios
    for title, price in zip(titles, prices):
        # Obtener el título y el precio y agregarlos a la lista
        titulo = title.text.strip()
        precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')  # Formatear el precio
        productos.append({'Título': titulo, 'Precio': float(precio)})

    
    # Cierra el navegador
    driver.quit()
    return productos

In [7]:
# Función para obtener la información de un producto en la página de Selma Digital
def obtener_info_selma_digital(nombre_producto):
    # URL de Selma Digital
    url_selma_digital = f'https://selmadigital.com/shop?search={nombre_producto}'

    # Realizar la solicitud HTTP
    response_selma_digital = requests.get(url_selma_digital)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_selma_digital.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_selma_digital = BeautifulSoup(response_selma_digital.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "product-card-design8-vertical__name"
        titles = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__name')

        # Buscar los elementos que contienen el precio con la clase "product-card-design8-vertical__price"
        prices = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__price')

        # Crear una lista para almacenar los productos
        productos = []

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace(',', '')  # Eliminar caracteres no numéricos del precio
            productos.append({'Título': titulo, 'Precio': float(precio)})

        return productos

    return None

In [19]:
#funcion para consultar en la pagina de openfarma
def obtener_info_openfarma(nombre_producto):
    # URL de Openfarma
    url_openfarma = f'https://www.openfarma.com.ar/products?utf8=%E2%9C%93&keywords={nombre_producto}'

    # Realizar la solicitud HTTP
    response_openfarma = requests.get(url_openfarma)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_openfarma.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_openfarma = BeautifulSoup(response_openfarma.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "card-title"
        titles = soup_openfarma.find_all('h2', class_='card-title')

        # Buscar los elementos que contienen el precio con la clase "regular"
        prices_regular = soup_openfarma.find_all('span', class_='regular')
        
        # Buscar los elementos que contienen el precio con la clase "promo"
        prices_promo = soup_openfarma.find_all('span', class_='promo')
        
        prices = prices_regular
        
        if len(prices_promo) > 0:
            prices = prices_promo
    

        # Crear una lista para almacenar los productos
        productos = []

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')  # Formatear el precio
            productos.append({'Título': titulo, 'Precio': float(precio)})

        return productos

    return None

In [9]:
# función para comparar precios consultados
def comparar_precios(productos_selma_digital, productos_openfarma):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')

In [10]:
def comparar_precios_3_paginas(productos_selma_digital, productos_openfarma, productos_farmacity):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}
    precios_farmacity = {producto['Título']: producto['Precio'] for producto in productos_farmacity}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_farmacity, precio_farmacity in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_farmacity)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_farmacity:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_farmacity:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_openfarma, precio_openfarma in precios_openfarma.items():
        for titulo_tercera, precio_tercera in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_openfarma, titulo_tercera)
            if similitud >= 80:  # Umbral de similitud
                if precio_openfarma < precio_tercera:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                elif precio_openfarma > precio_tercera:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_openfarma}": ${precio_openfarma}')

In [31]:
def comparar_precios_3_paginas_2(productos_selma_digital, productos_openfarma, productos_farmacity, umbral=80):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}
    precios_farmacity = {producto['Título']: producto['Precio'] for producto in productos_farmacity}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_farmacity, precio_farmacity in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_farmacity)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_selma < precio_farmacity:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_farmacity:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_openfarma, precio_openfarma in precios_openfarma.items():
        for titulo_tercera, precio_tercera in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_openfarma, titulo_tercera)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_openfarma < precio_tercera:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                elif precio_openfarma > precio_tercera:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_openfarma}": ${precio_openfarma}')

In [14]:
# Nombre del producto a buscar en ambas farmacias
nombre_producto = 'Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml'

In [28]:
# Obtener e imprimir la información de los productos de Selma Digital
productos_selma_digital = obtener_info_selma_digital(nombre_producto)
if productos_selma_digital:
    print(f'Productos encontrados en Selma Digital para "{nombre_producto}":')
    for idx, producto in enumerate(productos_selma_digital, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Selma Digital para "{nombre_producto}".')

# Obtener e imprimir la información de los productos de Openfarma
productos_openfarma = obtener_info_openfarma(nombre_producto)
if productos_openfarma:
    print(f'\nProductos encontrados en Openfarma para "{nombre_producto}":')
    for idx, producto in enumerate(productos_openfarma, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Openfarma para "{nombre_producto}".')


# Obtener e imprimir la información de los productos de Farmacity
productos_farmacity = obtener_info_farmacity(nombre_producto)
if productos_farmacity:
    print(f'\nProductos encontrados en Farmacity para "{nombre_producto}":')
    for idx, producto in enumerate(productos_farmacity, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Farmacity para "{nombre_producto}".')





Productos encontrados en Selma Digital para "Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml":
#1 - Título: Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml, Precio: $20.261
#2 - Título: Cicaplast Manos Crema Reparadora X 50 Ml, Precio: $32.15
#3 - Título: Combo Cicaplast Labios Balsamo Reparador + Cicaplast Crema Manos, Precio: $51.169
#4 - Título: Crema De Manos y Uñas x 50 Gr, Precio: $4.668
#5 - Título: Répaphénol Mains Crema De Manos Nutritiva Reparadora X 50 Gr, Precio: $9.807
#6 - Título: Combo X2 Crema De Manos 60 Ml, Precio: $22.0
#7 - Título: Combo X2 Crema Gel Ultra Hidratacion Para Manos 50gr, Precio: $11.007
#8 - Título: Combo X2 Crema Para Manos Hidratacion + Regeneracion 50gr, Precio: $11.007
#9 - Título: Combo X2 Crema Para Manos Hidratacion Esencial 100gr, Precio: $15.092
#10 - Título: Combo X2 Crema Para Manos Hidratacion Esencial 50gr, Precio: $8.875
#11 - Título: Combo X3 Crema Gel Ultra Hidratacion Para Manos 50gr, Precio: $15.333
#12 - Tí

In [32]:
# Comparar precios
if productos_selma_digital and productos_openfarma and productos_farmacity:
    comparar_precios_3_paginas_2(productos_selma_digital, productos_openfarma, productos_farmacity,umbral=75)

Selma Digital tiene un precio más bajo para "Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml": $20.261
Selma Digital tiene un precio más bajo para "Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml": $20.261
Selma Digital tiene un precio más bajo para "Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml": $20.261
Openfarma tiene un precio más bajo para "Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml": $19784.25
Farmacity tiene un precio más bajo para "Crema de Manos Antimanchas Eucerin Anti-Pigment con Fps 30 x 75 ml": $21826.0
